In [1]:
# import warnings
# warnings.filterwarnings('ignore')
import numpy as np
import cv2
from keras.models import load_model

In [2]:
# hàm nomarlize data
def preprocessing(img):
    img=img.astype("uint8")
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert sang màu xám để tăng hiệu năng thay vì để 3 màu
    img=cv2.equalizeHist(img) # cân bằng biểu đồ xám, cải thiện độ tương phản hình ảnh, mở rộng phạm vi cường độ
    img = img/255
    return img
# lấy tên của nhãn dán
def get_className(classNo):
    if classNo==0:
        return "Mask"
    elif classNo==1:
        return "No Mask"


In [3]:
# sử dụng file haarcascade_frontalface_default.xml đã tích hợp sẵn thuật toán)
# File này sẽ tích hợp sẵn các feature để nhận diện khuôn mặt
# Haar Cascade classifier ~ effective approach detection object
facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

threshold=0.90

# khởi tạo camera đầu tiên của máy tính, có index = 0
cap=cv2.VideoCapture(0)

# set parameter cho camera
cap.set(3, 640)
cap.set(4, 480)

font=cv2.FONT_HERSHEY_COMPLEX

#load model
model = load_model('MyTrainingModel.h5')

In [6]:
while True:
    sucess, imgOrignal=cap.read()
    # sucess có kiểu là boolean, dòng này có nghĩa nếu imgOrignal được đọc thành công nếu sucess = true(tức mở camera thành công)
    # khá là giống success&imgOrignal= cap.read()
    
    # trả về hình chữ nhật ranh giới cho khuôn mặt được phát hiện
    # có ý nghĩa sẽ dùng mọi feature của file haarcascade_frontalface_default.xml để phát hiện
    faces = facedetect.detectMultiScale(imgOrignal,1.3,5) #image/size image được giảm/
    # ở đây ta giảm tỉ lệ size của ảnh đi 1,3 lần để trong trường hợp khuôn mặt to sẽ giảm đi để có thể detect được khuôn mặt
    # điều này có thể khiến việc detect nhanh hơn, thuật toán chạy nhanh hơn tuy nhiên
    # Khi giảm tỉ lệ đi đồng nghĩa có thể bỏ sót 1 vài chi tiết của khuôn mặt do vậy không thể giảm quá nhiều
    # 5 là tham số chỉ đỉnh số lượng hàng xóm mà hình chữ nhật khuông mặt có(kiểu hình chữ nhật nhỏ xung quanh ấy :)) cũng không chắc nữa). Nó sẽ quyết định
    # đến chất lượng khi phát hiện ảnh. Giá trị càng cao thì ít bị phát hiện hơn nhưng chất lượng cao hơn. 5 là con số vừa đủ
    
    for x,y,w,h in faces: # x-cor, y-cor, width, height, dựa vào 4 tham số này để vẽ hình chữ nhật xung quanh khuôn mặt 
        crop_img=imgOrignal[y:y+h,x:x+w] # lấy ảnh chứa mặt, từ hàng y-> y+h, từ cột x->x+w
        
        # chuyển sang kênh màu xám vì kênh màu xám dễ xử lý hơn, và ít chuyên sâu về tính toán vì nó chỉ chứa 1 kênh đen trắng
        # resize 32x32
        img=cv2.resize(crop_img, (32,32)) 
        
        # normalize img
        img=preprocessing(img)
        
        # reshape lại
        img=img.reshape(1, 32, 32, 1) # 1 ảnh/32/32/kênh gray

        # tính prediction từ model
        prediction=model.predict(img)
        
        # lấy classIndex của prediction, prediction có dạng là một numpy array, np.argmax trả về index của giá trị lớn nhất
        # của mảng bắt đầu theo cột chỉ định là 1
        classIndex= np.argmax(prediction,axis=1)
        
        probabilityValue=np.amax(prediction) #trả về giá trị lớn nhất
        print(probabilityValue, " " , classIndex," ", prediction)
        if probabilityValue>threshold:
#             print(classIndex)
            if classIndex==1:
#                # vẽ giao diện
                cv2.rectangle(imgOrignal,(x,y),(x+w,y+h),(0,255,0),2) # điểm bắt đầu, điểm kết thúc (2 điểm của 2 góc hình chữ nhật), màu, độ dày của cạnh
                cv2.rectangle(imgOrignal, (x,y-40),(x+w, y), (0,255,0),-2)
                cv2.putText(imgOrignal, str(get_className(classIndex)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)
            elif classIndex==0:
#              
                cv2.rectangle(imgOrignal,(x,y),(x+w,y+h),(50,50,255),2)
                cv2.rectangle(imgOrignal, (x,y-40),(x+w, y), (50,50,255),-2)
                cv2.putText(imgOrignal, str(get_className(classIndex)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)
    cv2.imshow("Result",imgOrignal)
    
    k=cv2.waitKey(1) #đợi nhập key
    if k==ord('q'):# nhập q để break loop
        break


cap.release()
cv2.destroyAllWindows()

0.99997234   [0]   [[9.9997234e-01 2.7607550e-05]]
0.99314195   [0]   [[0.99314195 0.00685808]]
0.9570834   [0]   [[0.9570834 0.0429166]]
0.81045693   [0]   [[0.81045693 0.18954305]]
0.8315059   [0]   [[0.8315059  0.16849416]]
0.8755428   [0]   [[0.8755428  0.12445721]]
0.98945487   [0]   [[0.98945487 0.01054512]]
0.9913145   [0]   [[0.9913145  0.00868556]]
0.97569305   [0]   [[0.97569305 0.02430697]]
0.9466176   [0]   [[0.9466176  0.05338239]]
0.9474313   [0]   [[0.9474313  0.05256865]]
0.98650867   [0]   [[0.98650867 0.01349134]]
0.9907872   [0]   [[0.9907872  0.00921279]]
0.82189137   [0]   [[0.82189137 0.17810865]]
0.8024121   [0]   [[0.8024121 0.1975879]]
0.9623829   [0]   [[0.9623829  0.03761711]]
0.7323977   [0]   [[0.7323977  0.26760238]]
0.9840859   [0]   [[0.9840859  0.01591416]]
0.8903332   [0]   [[0.8903332  0.10966685]]
0.8975159   [0]   [[0.8975159  0.10248408]]
0.99507326   [0]   [[0.99507326 0.00492676]]
0.9529412   [0]   [[0.9529412  0.04705876]]
0.99610645   [0]   [[0

0.9946977   [1]   [[0.00530232 0.9946977 ]]
0.9666354   [1]   [[0.03336463 0.9666354 ]]
0.9835616   [1]   [[0.0164384 0.9835616]]
0.9857066   [1]   [[0.01429334 0.9857066 ]]
0.99084175   [1]   [[0.00915828 0.99084175]]
0.9747053   [1]   [[0.02529472 0.9747053 ]]
0.9846822   [1]   [[0.01531785 0.9846822 ]]
0.9804704   [1]   [[0.01952953 0.9804704 ]]
0.9845835   [1]   [[0.01541652 0.9845835 ]]
0.986485   [1]   [[0.01351499 0.986485  ]]
0.98293865   [1]   [[0.01706127 0.98293865]]
0.9824552   [1]   [[0.01754487 0.9824552 ]]
0.9799846   [1]   [[0.02001549 0.9799846 ]]
0.981685   [1]   [[0.01831508 0.981685  ]]
0.97940296   [1]   [[0.02059707 0.97940296]]
0.98880696   [1]   [[0.01119302 0.98880696]]
0.99164176   [1]   [[0.00835819 0.99164176]]
0.98866653   [1]   [[0.01133353 0.98866653]]
0.98865324   [1]   [[0.01134674 0.98865324]]
0.988123   [1]   [[0.01187696 0.988123  ]]
0.99114543   [1]   [[0.00885454 0.99114543]]
0.9862861   [1]   [[0.01371389 0.9862861 ]]
0.98916644   [1]   [[0.010833

0.9879488   [0]   [[0.9879488  0.01205122]]
0.9891132   [0]   [[0.9891132  0.01088683]]
0.9894231   [0]   [[0.9894231  0.01057691]]
0.99849737   [0]   [[0.99849737 0.00150261]]
0.9865262   [0]   [[0.9865262  0.01347379]]
0.99263906   [0]   [[0.99263906 0.00736092]]
0.99864346   [0]   [[0.99864346 0.00135659]]
0.995271   [0]   [[0.995271   0.00472901]]
0.99376523   [0]   [[0.99376523 0.00623477]]
0.9847388   [0]   [[0.9847388  0.01526111]]
0.9720072   [0]   [[0.9720072  0.02799282]]
0.9786669   [0]   [[0.9786669  0.02133309]]
0.99862766   [0]   [[0.99862766 0.00137231]]
0.98204076   [0]   [[0.98204076 0.01795924]]
0.99076754   [0]   [[0.99076754 0.0092324 ]]
0.9992888   [0]   [[9.9928880e-01 7.1114424e-04]]
0.9993967   [0]   [[9.9939668e-01 6.0330675e-04]]
0.9954385   [0]   [[0.9954385  0.00456153]]
0.991625   [0]   [[0.991625   0.00837504]]
0.98781186   [0]   [[0.98781186 0.01218814]]
0.9734302   [0]   [[0.9734302  0.02656979]]
0.99641526   [0]   [[0.99641526 0.00358469]]
0.9969893   [

0.9932636   [1]   [[0.00673638 0.9932636 ]]
0.98270226   [1]   [[0.01729768 0.98270226]]
0.97945297   [1]   [[0.02054703 0.97945297]]
0.98865265   [1]   [[0.01134736 0.98865265]]
0.99041384   [1]   [[0.00958616 0.99041384]]
0.9906198   [1]   [[0.00938018 0.9906198 ]]
0.9921227   [1]   [[0.00787733 0.9921227 ]]
0.9772003   [1]   [[0.02279966 0.9772003 ]]
0.99586594   [1]   [[0.00413402 0.99586594]]
0.99008334   [1]   [[0.00991667 0.99008334]]
0.9932308   [1]   [[0.00676919 0.9932308 ]]
0.9896271   [1]   [[0.01037289 0.9896271 ]]
0.9880229   [1]   [[0.01197709 0.9880229 ]]
0.99398065   [1]   [[0.00601936 0.99398065]]
0.99675566   [1]   [[0.00324431 0.99675566]]
0.9895611   [1]   [[0.01043892 0.9895611 ]]
0.9952982   [1]   [[0.00470182 0.9952982 ]]
0.9962519   [1]   [[0.00374818 0.9962519 ]]
0.9934947   [1]   [[0.00650529 0.9934947 ]]
0.99090403   [1]   [[0.00909599 0.99090403]]
0.9835332   [1]   [[0.01646683 0.9835332 ]]
0.98433137   [1]   [[0.01566864 0.98433137]]
0.98694384   [1]   [[0

0.99084574   [1]   [[0.00915426 0.99084574]]
0.99212027   [1]   [[0.00787979 0.99212027]]
0.9924563   [1]   [[0.0075437 0.9924563]]
0.9945175   [1]   [[0.00548245 0.9945175 ]]
0.9919152   [1]   [[0.00808478 0.9919152 ]]
0.9927174   [1]   [[0.00728257 0.9927174 ]]
0.9844379   [1]   [[0.01556208 0.9844379 ]]
0.98801607   [1]   [[0.01198401 0.98801607]]
0.9927241   [1]   [[0.00727592 0.9927241 ]]
0.9935916   [1]   [[0.0064084 0.9935916]]
0.99668294   [1]   [[0.00331701 0.99668294]]
0.98392534   [1]   [[0.01607461 0.98392534]]
0.98468393   [1]   [[0.01531612 0.98468393]]
0.9957681   [1]   [[0.00423196 0.9957681 ]]
0.99271905   [1]   [[0.00728098 0.99271905]]
0.99420494   [1]   [[0.00579512 0.99420494]]
0.99553865   [1]   [[0.00446138 0.99553865]]
0.99379784   [1]   [[0.00620209 0.99379784]]
0.9940386   [1]   [[0.00596147 0.9940386 ]]
0.9952513   [1]   [[0.00474875 0.9952513 ]]
0.99660707   [1]   [[0.00339297 0.99660707]]
0.99258184   [1]   [[0.0074182  0.99258184]]
0.9927153   [1]   [[0.00

0.5679284   [0]   [[0.5679284  0.43207166]]
0.93024075   [1]   [[0.06975926 0.93024075]]
0.96824336   [1]   [[0.0317566  0.96824336]]
0.97795606   [1]   [[0.02204398 0.97795606]]
0.87115914   [1]   [[0.12884094 0.87115914]]
0.7800957   [1]   [[0.21990424 0.7800957 ]]
0.6032651   [0]   [[0.6032651  0.39673492]]
0.78044283   [0]   [[0.78044283 0.21955717]]
0.7099768   [0]   [[0.7099768  0.29002324]]
0.6464223   [1]   [[0.35357764 0.6464223 ]]
0.9732512   [1]   [[0.02674882 0.9732512 ]]
0.9678628   [1]   [[0.0321372 0.9678628]]
0.93102545   [1]   [[0.06897453 0.93102545]]
0.84476197   [1]   [[0.15523802 0.84476197]]
0.9691815   [1]   [[0.0308185 0.9691815]]
0.9302046   [1]   [[0.0697954 0.9302046]]
0.83254313   [1]   [[0.16745682 0.83254313]]
0.5804362   [1]   [[0.4195638 0.5804362]]
0.9665113   [1]   [[0.03348861 0.9665113 ]]
0.56487155   [1]   [[0.43512848 0.56487155]]
0.63978404   [1]   [[0.36021602 0.63978404]]
0.78686386   [0]   [[0.78686386 0.21313612]]
0.5375999   [0]   [[0.5375999

0.51564693   [0]   [[0.51564693 0.48435304]]
0.7507037   [0]   [[0.7507037  0.24929628]]
0.74217486   [0]   [[0.74217486 0.2578252 ]]
0.6687851   [0]   [[0.6687851 0.3312149]]
0.7523399   [0]   [[0.7523399  0.24766013]]
0.56346595   [0]   [[0.56346595 0.4365341 ]]
0.5295281   [1]   [[0.47047192 0.5295281 ]]
0.726472   [0]   [[0.726472   0.27352798]]
0.6442102   [1]   [[0.3557898 0.6442102]]
0.64172614   [0]   [[0.64172614 0.35827386]]
0.7854713   [0]   [[0.7854713  0.21452874]]
0.8561185   [0]   [[0.8561185  0.14388146]]
0.8822046   [0]   [[0.8822046  0.11779549]]
0.8426016   [0]   [[0.8426016  0.15739843]]
0.7104416   [0]   [[0.7104416 0.2895584]]
0.6636229   [1]   [[0.33637702 0.6636229 ]]
0.9149822   [0]   [[0.9149822  0.08501777]]
0.9480651   [0]   [[0.9480651  0.05193489]]
0.9359344   [0]   [[0.9359344  0.06406555]]
0.6718111   [1]   [[0.32818893 0.6718111 ]]
0.8633307   [0]   [[0.8633307  0.13666932]]
0.6232533   [1]   [[0.37674665 0.6232533 ]]
0.79422385   [0]   [[0.79422385 0.2

0.7455635   [1]   [[0.25443646 0.7455635 ]]
0.93768847   [1]   [[0.06231153 0.93768847]]
0.9780734   [1]   [[0.02192659 0.9780734 ]]
0.9448422   [1]   [[0.05515775 0.9448422 ]]
0.72498614   [1]   [[0.27501386 0.72498614]]
0.7446457   [1]   [[0.2553543 0.7446457]]
0.6750427   [1]   [[0.32495728 0.6750427 ]]
0.72922784   [1]   [[0.27077225 0.72922784]]
0.9656323   [1]   [[0.03436775 0.9656323 ]]
0.9340474   [1]   [[0.06595257 0.9340474 ]]
0.9392464   [1]   [[0.06075359 0.9392464 ]]
0.50891155   [0]   [[0.50891155 0.49108845]]
0.5509733   [1]   [[0.44902676 0.5509733 ]]
0.6829694   [1]   [[0.3170306 0.6829694]]
0.6390384   [1]   [[0.36096165 0.6390384 ]]
0.82153225   [1]   [[0.17846775 0.82153225]]
0.7543777   [1]   [[0.24562223 0.7543777 ]]
0.9363427   [1]   [[0.06365737 0.9363427 ]]
0.88465154   [1]   [[0.11534852 0.88465154]]
0.9566326   [1]   [[0.04336739 0.9566326 ]]
0.9773356   [1]   [[0.02266438 0.9773356 ]]
0.91019696   [1]   [[0.08980304 0.91019696]]
0.8367311   [1]   [[0.1632689

0.9967501   [0]   [[0.9967501  0.00324992]]
0.99915814   [0]   [[9.9915814e-01 8.4183732e-04]]
0.99787056   [0]   [[0.99787056 0.00212937]]
0.99920684   [0]   [[9.9920684e-01 7.9319382e-04]]
0.9983602   [0]   [[0.9983602  0.00163978]]
0.9935638   [0]   [[0.9935638  0.00643623]]
0.9987669   [0]   [[0.9987669  0.00123314]]
0.99661154   [0]   [[0.99661154 0.00338849]]
0.99923575   [0]   [[9.9923575e-01 7.6427677e-04]]
0.99691594   [0]   [[0.99691594 0.00308404]]
0.9968836   [0]   [[0.9968836 0.0031164]]
0.9985989   [0]   [[0.9985989  0.00140113]]
0.9979607   [0]   [[0.9979607  0.00203927]]
0.9943534   [0]   [[0.9943534 0.0056466]]
0.99919504   [0]   [[9.9919504e-01 8.0492540e-04]]
0.99982315   [0]   [[9.9982315e-01 1.7686399e-04]]
0.99627185   [0]   [[0.99627185 0.00372817]]
0.9984944   [0]   [[0.9984944  0.00150562]]
0.9980538   [0]   [[0.9980538  0.00194627]]
0.98541313   [0]   [[0.98541313 0.0145869 ]]
0.99412894   [0]   [[0.99412894 0.00587109]]
0.9873377   [0]   [[0.9873377  0.012662

KeyboardInterrupt: 